# Units

In [ ]:
"""
Lightweight, fast units + conversion framework (so far).

Design principles:
- Each physical quantity owns its own Units (IntEnum) and conversion factors to a canonical base unit.
- Conversions are stateless: float/ndarray in -> float/ndarray out.
- Unit systems (UnitsSI/UnitsCGS/...) are pure namespaces selecting canonical units per quantity.
- No parsing, no dimensional algebra (keeps it fast vs Pint).
"""

from enum import IntEnum


# -----------------------------------------------------------------------------
# Pressure
# Canonical base unit: Pa
# -----------------------------------------------------------------------------
class Pressure:
    class Units(IntEnum):
        Pa   = 0
        kPa  = 1
        MPa  = 2
        GPa  = 3
        bar  = 4
        atm  = 5
        mbar = 6
        hPa  = 7
        Torr = 8
        mmHg = 9
        psi  = 10
        Ba   = 11   # CGS barye = dyne/cm^2
        cmH2O = 12  # conventional

    # factor to convert *to Pa* (tuple index matches Units)
    _TO_PA = (
        1.0,                 # Pa
        1e3,                 # kPa
        1e6,                 # MPa
        1e9,                 # GPa
        1e5,                 # bar
        101_325.0,           # atm
        100.0,               # mbar
        100.0,               # hPa
        101_325.0 / 760.0,   # Torr (by convention)
        101_325.0 / 760.0,   # mmHg (pedagogical equivalence)
        6894.757293168,      # psi
        0.1,                 # Ba (barye) = 0.1 Pa
        98.0665,             # cmH2O (conventional reference)
    )

    @staticmethod
    def convert(*, from_, to: "Pressure.Units"):
        """
        Convert pressure: from_ = [value, unit_from], to = unit_to.
        value can be a float OR a NumPy array (vectorized).
        """
        value, u_from = from_
        return value * (Pressure._TO_PA[int(u_from)] / Pressure._TO_PA[int(to)])

    @staticmethod
    def to_base(value, unit: "Pressure.Units"):
        """Convert to base (Pa)."""
        return value * Pressure._TO_PA[int(unit)]

    @staticmethod
    def from_base(value_pa, unit: "Pressure.Units"):
        """Convert from base (Pa) to target."""
        return value_pa / Pressure._TO_PA[int(unit)]


# -----------------------------------------------------------------------------
# Force
# Canonical base unit: N
# -----------------------------------------------------------------------------
class Force:
    class Units(IntEnum):
        N    = 0
        kN   = 1
        dyne = 2  # CGS force

    # factor to convert *to N*
    _TO_N = (
        1.0,     # N
        1e3,     # kN
        1e-5,    # dyne = 1e-5 N
    )

    @staticmethod
    def convert(*, from_, to: "Force.Units"):
        """
        Convert force: from_ = [value, unit_from], to = unit_to.
        value can be a float OR a NumPy array (vectorized).
        """
        value, u_from = from_
        return value * (Force._TO_N[int(u_from)] / Force._TO_N[int(to)])

    @staticmethod
    def to_base(value, unit: "Force.Units"):
        """Convert to base (N)."""
        return value * Force._TO_N[int(unit)]

    @staticmethod
    def from_base(value_n, unit: "Force.Units"):
        """Convert from base (N) to target."""
        return value_n / Force._TO_N[int(unit)]


# -----------------------------------------------------------------------------
# Unit Systems (cross-quantity namespaces)
# These select canonical units per quantity.
# No math here. No conversion factors. Pure references.
# -----------------------------------------------------------------------------
class UnitsSI:
    pressure = Pressure.Units.Pa
    force    = Force.Units.N
    # length   = Length.Units.m
    # mass     = Mass.Units.kg
    # time     = Time.Units.s
    # energy   = Energy.Units.J


class UnitsCGS:
    pressure = Pressure.Units.Ba     # barye = dyne/cm^2
    force    = Force.Units.dyne
    # length   = Length.Units.cm
    # mass     = Mass.Units.g
    # time     = Time.Units.s


class UnitsImperial:
    pressure = Pressure.Units.psi
    # force    = Force.Units.lbf   # add when you define lbf
    # length   = Length.Units.inch # add when you define inch/ft
    # mass     = Mass.Units.slug   # optional
    # time     = Time.Units.s

class UnitsUSCS:
    force = Force.Units.lbf
    #mass  = Mass.Units.lbm   # requires g_c in equations
    #length = Length.Units.ft
    #time   = Time.Units.s


class UnitsVacuum:
    pressure = Pressure.Units.Torr
    # (vacuum work often uses Torr / mbar / Pa)

